In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from typing import *
import numpy as np
from functools import partial
from fastprogress import progress_bar
import pandas as pd
import h5py

from lumin.plotting.results import plot_roc

import torch
from torch import Tensor, nn
import torch.nn.functional as F
from torch._vmap_internals import _vmap as vmap

from tomopt.volume import *
from tomopt.muon import *
from tomopt.inference import *
from tomopt.optimisation import *
from tomopt.core import *
from tomopt.utils import *
from tomopt.plotting import *

import seaborn as sns
import matplotlib.pyplot as plt

/Users/giles/anaconda3/envs/tomopt/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
/Users/giles/anaconda3/envs/tomopt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
DEVICE = torch.device("cpu")
RES = 1e4
SPAN = 0.8

In [46]:
def get_central_separated_detector(size: float = 0.1, lwh: Tensor = Tensor([1.0, 1.0, 1.8]), span=SPAN, device: torch.device = torch.device("cpu")) -> Volume:
    def area_cost(x: Tensor) -> Tensor:
        return F.relu(x)

    layers: List[Layer] = []
    layers.append(
        PanelDetectorLayer(
            pos="above",
            lw=lwh[:2],
            z=lwh[2].item(),
            size=0.4,
            panels=[
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.8),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.75),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.5),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.45),
                    init_xy_span=(span, span),
                    device=device,
                ),
            ],
        )
    )
    for z in np.round(np.arange(lwh[2] - 0.4, 0.4, -size), decimals=2):
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=device))
    layers.append(
        PanelDetectorLayer(
            pos="below",
            lw=lwh[:2],
            z=0.2,
            size=0.4,
            panels=[
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.4),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.35),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.1),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.05),
                    init_xy_span=(span, span),
                    device=device,
                ),
            ],
        )
    )

    return Volume(nn.ModuleList(layers))

In [47]:
get_central_separated_detector()

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.8000]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.7500]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.5000]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.4500]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([1.4000])
    (2): PassiveLayer located at z=tensor([1.3000])
    (3): PassiveLayer loc

In [48]:
def get_central_close_detector(size: float = 0.1, lwh: Tensor = Tensor([1.0, 1.0, 1.8]), span=SPAN, device: torch.device = torch.device("cpu")) -> Volume:
    def area_cost(x: Tensor) -> Tensor:
        return F.relu(x)

    layers: List[Layer] = []
    layers.append(
        PanelDetectorLayer(
            pos="above",
            lw=lwh[:2],
            z=lwh[2].item(),
            size=0.4,
            panels=[
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.48),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.47),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.46),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 1.45),
                    init_xy_span=(span, span),
                    device=device,
                ),
            ],
        )
    )
    for z in np.round(np.arange(lwh[2] - 0.4, 0.4, -size), decimals=2):
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=device))
    layers.append(
        PanelDetectorLayer(
            pos="below",
            lw=lwh[:2],
            z=0.2,
            size=0.4,
            panels=[
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.4),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.39),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.38),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.37),
                    init_xy_span=(span, span),
                    device=device,
                ),
            ],
        )
    )

    return Volume(nn.ModuleList(layers))

In [49]:
get_central_close_detector()

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.4800]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.4700]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.4600]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.4500]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([1.4000])
    (2): PassiveLayer located at z=tensor([1.3000])
    (3): PassiveLayer loc

In [50]:
def get_offset_separated_detector(size: float = 0.1, lwh: Tensor = Tensor([1.0, 1.0, 1.8]), span=SPAN, device: torch.device = torch.device("cpu")) -> Volume:
    def area_cost(x: Tensor) -> Tensor:
        return F.relu(x)

    layers: List[Layer] = []
    layers.append(
        PanelDetectorLayer(
            pos="above",
            lw=lwh[:2],
            z=lwh[2].item(),
            size=0.4,
            panels=[
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(0.0, 0.0, 1.8),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(0.0, 0.0, 1.75),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(0.0, 0.0, 1.5),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(0.0, 0.0, 1.45),
                    init_xy_span=(span, span),
                    device=device,
                ),
            ],
        )
    )
    for z in np.round(np.arange(lwh[2] - 0.4, 0.4, -size), decimals=2):
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=device))
    layers.append(
        PanelDetectorLayer(
            pos="below",
            lw=lwh[:2],
            z=0.2,
            size=0.4,
            panels=[
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.4),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.35),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.1),
                    init_xy_span=(span, span),
                    device=device,
                ),
                SigmoidDetectorPanel(
                    smooth=0.1,
                    res=RES,
                    eff=1,
                    init_xyz=(lwh[0].item()/2, lwh[1].item()/2, 0.05),
                    init_xy_span=(span, span),
                    device=device,
                ),
            ],
        )
    )

    return Volume(nn.ModuleList(layers))

In [51]:
get_offset_separated_detector()

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0., 0.]), z=tensor([1.8000]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (1): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0., 0.]), z=tensor([1.7500]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (2): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0., 0.]), z=tensor([1.5000]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
        (3): <class 'tomopt.volume.panel.SigmoidDetectorPanel'> located at xy=tensor([0., 0.]), z=tensor([1.4500]), and xy span tensor([0.8000, 0.8000]) with budget scale tensor([1.])
      )
    )
    (1): PassiveLayer located at z=tensor([1.4000])
    (2): PassiveLayer located at z=tensor([1.3000])
    (3): PassiveLayer located at z=tensor([1.2000])
    (

In [52]:
from tomopt.benchmarks.ladle_furnace import *

In [53]:
from pathlib import Path

class PocaRec(PredHandler):
    def __init__(self, path: Path, overwrite:bool):
        if isinstance(path, str):
            path = Path(path)
        self.path = path
        if self.path.exists() and overwrite:
            self.path.unlink()
        self.path.parent.mkdir(exist_ok=True, parents=True)
        self.id = 0

    def _open_file(self) -> h5py.File:
        r"""
        Returns:
            Save file to write data to
        """

        if self.path.exists():
            return h5py.File(self.path, "r+")
        return h5py.File(self.path, "w")
                
    def on_x0_pred_end(self) -> None:
        r"""
        Records predictions and true volume layout or target for the latest volume
        """

        if self.wrapper.fit_params.state == "test":
            pred_h = self.wrapper.fit_params.pred.detach().cpu().numpy()
            targ_h = self.wrapper.volume.target.detach().cpu().numpy().item()
            pocas = self.wrapper.fit_params.volume_inferrer.muon_poca_xyz.detach().cpu().numpy()
            poca_uncs = self.wrapper.fit_params.volume_inferrer.muon_poca_xyz_unc.detach().cpu().numpy()
            eff = self.wrapper.fit_params.volume_inferrer.muon_efficiency.reshape(self.wrapper.fit_params.volume_inferrer.n_mu, 1).detach().cpu().numpy()
            sig_wgt = self.wrapper.fit_params.volume_inferrer.sig_wgt.detach().cpu().numpy()
            wgt = self.wrapper.fit_params.volume_inferrer.wgt.detach().cpu().numpy()
            
            with self._open_file() as h5:
                grp = h5.create_group(f'targ_h_{targ_h:.3f}_{self.id}')
                grp.create_dataset("pred_h", data=pred_h.astype("float32"), dtype="float32", compression=None)
                grp.create_dataset("targ_h", data=targ_h, dtype="float32", compression=None)                
                grp.create_dataset("poca_xyz", data=pocas.astype("float32"), dtype="float32", compression='lzf')      
                grp.create_dataset("poca_xyz_unc", data=poca_uncs.astype("float32"), dtype="float32", compression='lzf')               
                grp.create_dataset("muon_efficiency", data=eff.astype("float32"), dtype="float32", compression='lzf')               
                grp.create_dataset("muon_xy_sig_wgt", data=sig_wgt.astype("float32"), dtype="float32", compression='lzf')               
                grp.create_dataset("muon_wgt", data=wgt.astype("float32"), dtype="float32", compression='lzf')        
                
            self.id += 1

- Detectors:
    - Central, separated
    - Central, close
    - Offset, separated
- Muons:
    - 1000
    - 10000
    - 100000
- Fill-height:
    - 0.4
    - 0.6
    - 0.8
    - 1.0
    - 1.2

In [54]:
fill_heights = [0.4, 0.6, 0.8, 1.0, 1.2]*10

## No slag

In [56]:
%%time
for det, volume in [('central_separated', get_central_separated_detector()), ('central_close', get_central_close_detector()), ('offset_separated', get_offset_separated_detector())]:
    print('Running volume', det)
    passive_generator = LadleFurnacePassiveGenerator(volume)
    test_passives = PassiveYielder([passive_generator._generate(fixed_mat_z=h, fixed_slag_z=0.0) for h in fill_heights], shuffle=False)
    wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VolumeMSELoss(target_budget=None),
                             partial_volume_inferrer=partial(PocaZLadleFurnaceFillLevelInferrer, smooth=1.0))
    for n_mu in [1000, 10000]:
        print('Running muons', n_mu)
        wrapper.predict(test_passives,
                n_mu_per_volume=n_mu,
                mu_bs=250,
                cbs=[MuonResampler()],
                pred_cb=PocaRec(Path(f'ladle_inf_data_multi_thinspan_smooth_high/no_slag/det-{det}_n_mu-{n_mu}.h5'), True))

Running volume central_separated
Running muons 10000


Running volume central_close
Running muons 10000


KeyboardInterrupt: 

## Random slag

%%time
for det, volume in [('central_separated', get_central_separated_detector()), ('central_close', get_central_close_detector()), ('offset_separated', get_offset_separated_detector())]:
    print('Running volume', det)
    passive_generator = LadleFurnacePassiveGenerator(volume)
    test_passives = PassiveYielder([passive_generator._generate(fixed_mat_z=h) for h in fill_heights], shuffle=False)
    wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VolumeMSELoss(target_budget=None),
                             partial_volume_inferrer=partial(PocaZLadleFurnaceFillLevelInferrer))
    for n_mu in [1000, 10000, 100000]:
        print('Running muons', n_mu)
        wrapper.predict(test_passives,
                n_mu_per_volume=n_mu,
                mu_bs=250,
                cbs=[MuonResampler()],
                pred_cb=PocaRec(Path(f'ladle_inf_data/random_slag/det-{det}_n_mu-{n_mu}.h5'), True))

## 10cm slag

%%time
for det, volume in [('central_separated', get_central_separated_detector()), ('central_close', get_central_close_detector()), ('offset_separated', get_offset_separated_detector())]:
    print('Running volume', det)
    passive_generator = LadleFurnacePassiveGenerator(volume)
    test_passives = PassiveYielder([passive_generator._generate(fixed_mat_z=h, fixed_slag_z=h+0.1001) for h in fill_heights[:-1]], shuffle=False)
    wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VolumeMSELoss(target_budget=None),
                             partial_volume_inferrer=partial(PocaZLadleFurnaceFillLevelInferrer))
    for n_mu in [1000, 10000, 100000]:
        print('Running muons', n_mu)
        wrapper.predict(test_passives,
                n_mu_per_volume=n_mu,
                mu_bs=250,
                cbs=[MuonResampler()],
                pred_cb=PocaRec(Path(f'ladle_inf_data/10cm_slag/det-{det}_n_mu-{n_mu}.h5'), True))

## Rest slag

%%time
for det, volume in [('central_separated', get_central_separated_detector()), ('central_close', get_central_close_detector()), ('offset_separated', get_offset_separated_detector())]:
    print('Running volume', det)
    passive_generator = LadleFurnacePassiveGenerator(volume)
    test_passives = PassiveYielder([passive_generator._generate(fixed_mat_z=h, fixed_slag_z=1.4) for h in fill_heights[:-1]], shuffle=False)
    wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VolumeMSELoss(target_budget=None),
                             partial_volume_inferrer=partial(PocaZLadleFurnaceFillLevelInferrer))
    for n_mu in [1000, 10000, 100000]:
        print('Running muons', n_mu)
        wrapper.predict(test_passives,
                n_mu_per_volume=n_mu,
                mu_bs=250,
                cbs=[MuonResampler()],
                pred_cb=PocaRec(Path(f'ladle_inf_data/rest_slag/det-{det}_n_mu-{n_mu}.h5'), True))